In [1]:
import numpy as np 
import pygame 

from formula_functions import *
from animation_functions import *
from intersection_functions import create_spiral, distance
from show_parameters_functions import *

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Global animation variables

""" 

screen_width = 1500
screen_height = 800
FPS = 25

In [13]:
def main_loop(screen_width, screen_height, FPS):
    '''
    Initializations
    '''
    
    pygame.init()

    font_small = pygame.font.Font(None, 26)

    bg_color = (255, 255, 255)
    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer
    background_surface = pygame.Surface((screen_width, screen_height))
   
    line_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    spiral_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    params_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    

    background_surface.fill(bg_color)
    line_layer.fill((0,0,0,0))
    spiral_layer.fill((0, 0, 0, 0))
    params_layer.fill((0, 0, 0, 0))

    
    '''
    Global variables
    '''
    
    units =20
    half_units = units/2
    length = screen_width/ units
  
    v_step = 0.01
    w_step = 0.01
    k_step = 0.02
    x_step = 0.02
    
    v = 1
    w = 1
    k = 0
    vert_line_x = 1
    
    rotation_t = 2 * np.pi/(abs(w))
    
    one_rotation_rad_vec_len = v * rotation_t
    
    turns_count = half_units / one_rotation_rad_vec_len
    
    
    max_spiral_vector_length = np.sqrt(screen_width ** 2 + screen_height ** 2)/length
    
    const_params_dict = {'v': v, 'w': w, 'k': k, 'x': vert_line_x}
    
    v_additional, w_additional, x_additional, k_additional = 0, 0, 0, 0
    
    var_params_dict = {'v': v_additional, 
                       'w': w_additional, 
                       'k': k_additional, 
                       'x': x_additional}
    
    layers_list = [background_surface, line_layer, spiral_layer, params_layer]
    
    
    # Create background coordinate system
    create_background(background_surface, screen_width, screen_height, units, length, bg_color, font_small)
    
    x, y = x_transform(0, screen_width, length), y_transform(6, screen_height, length)

    draw_vertical_line(line_layer, screen_width, screen_height, length, half_units)
    
    draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)
    
    show_parameters(params_layer, font_small, const_params_dict,
                    var_params_dict)
    
    
    # Atach layers to screen
    blit_layers(win, layers_list, bg_color)
    
    pygame.display.update()
      
      
    running = True
    
    

    while running:
        
        clock.tick(FPS)
        
        udpate_screen = False
      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            if event.type == pygame.KEYDOWN and (pygame.key.get_mods() & pygame.KMOD_SHIFT) and event.key == pygame.K_w:

                w *= -1

                w_additional *= -1

                const_params_dict['w'] = w


                draw_spiral(spiral_layer, screen_width, screen_height,
                            length,max_spiral_vector_length, turns_count, 
                            v=v+v_additional,w= w+w_additional, k= k + k_additional)

                udpate_screen = True
        #Check for key combinations
        keys = pygame.key.get_pressed()

        # Left and right movement of the line
        if keys[pygame.K_x] and keys[pygame.K_RIGHT]:

            x_additional += x_step           

            draw_vertical_line(line_layer, screen_width, screen_height, length, half_units, x_axis_value=vert_line_x + x_additional)

            udpate_screen = True

        elif keys[pygame.K_x] and keys[pygame.K_LEFT]:
            x_additional -= x_step

            draw_vertical_line(line_layer, screen_width, screen_height, length, half_units, x_axis_value=vert_line_x + x_additional)

            udpate_screen = True

        # Increase angluar velocity `w`    
        elif keys[pygame.K_w] and keys[pygame.K_UP]:
            w_additional += w_step

            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        # Decrease angluar velocity `w`  
        elif keys[pygame.K_w] and keys[pygame.K_DOWN]:
            w_additional -= w_step
           

            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)
            udpate_screen = True
    
        # Increase initial spiral angle coefficient `k`  
        elif keys[pygame.K_k] and keys[pygame.K_UP]:
            k_additional += k_step
            new_k = k + k_additional

            if new_k >= 4:         
                k_additional = 0

            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True
           
        
        # Decrease initial spiral angle coefficient `k`
        elif keys[pygame.K_k] and keys[pygame.K_DOWN]:
            k_additional -= k_step
            new_k = k + k_additional

            if new_k <= 0:         
                k_additional = 4


            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        # Increase spiral radius velocity `v`
        elif keys[pygame.K_v] and keys[pygame.K_UP]:
            v_additional += v_step            

            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        # Decrease spiral radius velocity `v`
        elif keys[pygame.K_v] and keys[pygame.K_DOWN]:
            v_additional -= v_step
           

            if v + v_additional <= 0:
                v_additional = -v_step

            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        # Return to initial parameters
        elif (keys[pygame.K_LSHIFT] or keys[pygame.K_RSHIFT]) and keys[pygame.K_r]:

            v_additional, w_additional, x_additional, k_additional = 0, 0, 0, 0
            
            draw_vertical_line(line_layer, screen_width, screen_height, length, half_units, x_axis_value=vert_line_x + x_additional)

            draw_spiral(spiral_layer, screen_width, screen_height,
                        length,max_spiral_vector_length, turns_count, 
                        v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True
            
            
        if udpate_screen:

            var_params_dict = update_var_params_dict(var_params_dict, v_additional, w_additional, x_additional, k_additional)


            show_parameters(params_layer, font_small, const_params_dict,
                            var_params_dict)  


            blit_layers(win, layers_list, bg_color)


            pygame.display.update()

        
    # Exit
    pygame.quit()
    

In [14]:
main_loop(screen_width, screen_height, FPS)

In [5]:
a = 0
b = 1.5
lim = 40

# print(np.linspace(a,b,lim))

unit=(b-a)/(lim-1)
# print('unit: ', unit)
ll = []
for i in range(lim):
    ll.append(round(unit*i,8))
# print('ll')
# print(np.array(ll))

In [6]:
# radian_unit = 2 * np.pi/360
# length = screen_width/ 20

# spiral_coords = []
# distances = []

# v = 0.1
# w = 0.5
# k = 0
# theta_0 = k * np.pi/2
# for i in range(30*361):
#     angle = radian_unit*i
#     t = angle/abs(w)
#     x_spiral = v * t * np.cos(theta_0 + w * t)
#     y_spiral = v * t * np.sin(theta_0 + w * t)
    
#     x_transformed = x_transform(x_spiral, screen_width, length)
#     y_transformed = y_transform(y_spiral, screen_height, length)
    
    
#     spiral_coords.append([x_transformed, y_transformed])
#     if len(spiral_coords) > 1:
#         dist = distance(spiral_coords[-1], spiral_coords[-2])
#         distances.append(int(np.ceil(dist)))
        
# # print(distances, sum(distances))    